In [1]:
# default_exp network

# python

In [2]:
import util

# node

In [3]:
%%node export

import buffer from "buffer"

export var encode = (value) => {
    return new Uint8Array(buffer.Buffer.from(value, "base64"))
}

import algosdk from "algosdk"

export var newClient = (token, server, port) => {
    return new algosdk.Algodv2(token, server, port)
}

export var deployApplication = async (
    client, 
    address, 
    privateKey, 
    applicationStartTeal, 
    clearProgramTeal, 
    options = {
        numGlobalByteSlices: 0,
        numGlobalInts: 0,
        numLocalByteSlices: 0,
        numLocalInts: 0
    }) => {
    try {
        const suggestedParams = await client.getTransactionParams().do()
        suggestedParams.flatFee = true
        suggestedParams.fee = 1000

        const applicationStartCompiled = await client.compile(applicationStartTeal).do()
        const clearProgramCompiled = await client.compile(clearProgramTeal).do()

        const txn = algosdk.makeApplicationCreateTxnFromObject({
            suggestedParams,
            from: address,
            onComplete: algosdk.OnApplicationComplete.NoOpOC,
            approvalProgram: encode(applicationStartCompiled.result),
            clearProgram: encode(clearProgramCompiled.result),
            ...options
        })

        const txnSigned = txn.signTxn(encode(privateKey))
        const { txId } = await client.sendRawTransaction(txnSigned).do()
        const transactionResponse = await algosdk.waitForConfirmation(client, txId, 5)
        const appId = transactionResponse["application-index"]

        return {
            suggestedParams,
            applicationStartCompiled,
            clearProgramCompiled,
            txn,
            txnSigned,
            txId,
            transactionResponse,
            appId
        }
    } catch (e) {
        console.log('error (rethrowing):', e.toString())
        throw e
    }
}


e9eadce44b82437ba1c8a4005162a6bd


<IPython.core.display.Javascript object>

'network.ts'

In [4]:
%%node run

var token = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
var server = `http://${process.env.CODESPACE_NAME}-4001.githubpreview.dev`
var port = 80

var client = newClient(token, server, port)
var status = await client.status().do()
console.log({ status })


['{',
 '  status: {',
 "    catchpoint: '',",
 "    'catchpoint-acquired-blocks': 0,",
 "    'catchpoint-processed-accounts': 0,",
 "    'catchpoint-total-accounts': 0,",
 "    'catchpoint-total-blocks': 0,",
 "    'catchpoint-verified-accounts': 0,",
 "    'catchup-time': 0,",
 "    'last-catchpoint': '',",
 "    'last-round': 21962822,",
 "    'last-version': 'https://github.com/algorandfoundation/specs/tree/d5ac876d7ede07367dbaa26e149aa42589aac1f7',",
 "    'next-version': 'https://github.com/algorandfoundation/specs/tree/d5ac876d7ede07367dbaa26e149aa42589aac1f7',",
 "    'next-version-round': 21962823,",
 "    'next-version-supported': true,",
 "    'stopped-at-unsupported-round': false,",
 "    'time-since-last-round': 3678078227",
 '  }',
 '}']